# Pain Ranks - Yahoo Fin

Ranks input stocks and ETFs based on the 


In [1]:
!pip install yahoo_fin
!pip install tabulate
!pip install bokeh
!pip install panel
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [2]:
### # G

from yahoo_fin import options as op
from yahoo_fin import stock_info
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import math

pd.set_option('display.max_rows', 2000)

def get_max_pain(ticker):

    expirationDates = op.get_expiration_dates(ticker)

    # HARDCODED right now to use first expiration cycle
    # todo reduce to one "get options data" api call?
    callData = op.get_calls(ticker, date = expirationDates[0])
    putData = op.get_puts(ticker, date = expirationDates[0])
    
    callData.drop('Contract Name', axis=1, inplace=True) 
    callData.drop('Bid', axis=1, inplace=True)
    callData.drop('Last Trade Date', axis=1, inplace=True) 
    callData.drop('Change', axis=1, inplace=True) 
    callData.drop('% Change', axis=1, inplace=True) 
    callData.drop('Implied Volatility', axis=1, inplace=True) 
    callData.drop('Last Price', axis=1, inplace=True) 

    putData.drop('Contract Name', axis=1, inplace=True) 
    putData.drop('Bid', axis=1, inplace=True) 
    putData.drop('Last Trade Date', axis=1, inplace=True) 
    putData.drop('Change', axis=1, inplace=True) 
    putData.drop('% Change', axis=1, inplace=True) 
    putData.drop('Implied Volatility', axis=1, inplace=True) 
    putData.drop('Last Price', axis=1, inplace=True) 

    print("\n\nCall data for ", ticker)
    print("ex: ", expirationDates[0])
    print(callData)
    print("\n\nPut data for ", ticker)
    print(putData)
    
    option_chain_data_list = []   

    cash_calcs_list = []

    showCalcsCheckbox = widgets.Checkbox(
        value=False,
        description='Show Intermediate T',
        disabled=False
    )
    
    # dictionary that maps each "potential closing price" to the payoff for calls and puts    
    total_options_payoff_for_close = {}
    call_options_payoff_for_close = {}
    put_options_payoff_for_close = {}
    
    # HARDCODED for treating 100 percent volume as newly opened contracts
    volume_percentage = 0

    total_options_payoff_by_close = {}

    # Call Payoffs #2

    call_payoffs = {} 
    for index, strike_row in callData.iterrows():

        running_total_for_strike = 0
        strike = int(strike_row['Strike'])
        
        for inner_index, inner_strike_row in callData.iterrows():

            inner_strike = int(inner_strike_row['Strike'])
            
            if (inner_strike < strike):

                inner_oi = 0 if inner_strike_row['Open Interest'] == '-' else int(inner_strike_row['Open Interest'])

                if (inner_oi > 0):

                    payoff_these_two_strikes = (strike - inner_strike) * inner_oi * 100
                    running_total_for_strike += payoff_these_two_strikes

            call_payoffs[strike] = running_total_for_strike

            if (strike not in total_options_payoff_by_close):
                total_options_payoff_by_close[strike] = 0
                
            total_options_payoff_by_close[strike] += running_total_for_strike


    print ("call cash")
    print ("Strike | Cash")
    
    print(len(call_payoffs.items()))
    for (key, value) in call_payoffs.items():
        print (key, value)
        
    # ==============================================================

     # Put Payoffs #2

    put_payoffs = {} 
    for index, strike_row in putData.iterrows():

        running_total_for_strike = 0
        strike = int(strike_row['Strike'])
        
        for inner_index, inner_strike_row in putData.iterrows():

            inner_strike = int(inner_strike_row['Strike'])
            
            if (inner_strike > strike):

                inner_oi = 0 if inner_strike_row['Open Interest'] == '-' else int(inner_strike_row['Open Interest'])

                if (inner_oi > 0):

                    payoff_these_two_strikes = (inner_strike - strike) * inner_oi * 100
                    running_total_for_strike += payoff_these_two_strikes

            put_payoffs[strike] = running_total_for_strike
            
            if (strike not in total_options_payoff_by_close):
                total_options_payoff_by_close[strike] = 0
                
            total_options_payoff_by_close[strike] += running_total_for_strike

    print ("put cash")
    print ("Strike | Cash")
    
    print(len(call_payoffs.items()))
    for (key, value) in put_payoffs.items():
        print (key, value)
        
    # ==============================================================

    print ("Puts And Calls Cash")
    print ("Strike | Cash")
    print(len(call_payoffs.items()))
    for (key, value) in total_options_payoff_by_close.items():
        print (key, value)
        
    # ==============================================================
    

    # Build Intermediate Calcs Table
    
    option_calcs_calls_list = []
    option_calcs_puts_list = []
    option_calcs_totals_list = []

    print("\n\n", ticker, " Calls + Puts Calcs")
    for (close, payoff) in total_options_payoff_by_close.items():
        calcs_dic = {}
        calcs_dic.update({"Strike": close})
        calcs_dic.update({"Total Cash": payoff})
        option_calcs_totals_list.append(calcs_dic)
    
    option_calcs_totals_df = pd.DataFrame(option_calcs_totals_list)
    
    print(option_calcs_totals_df)
    
    # ===============================================================
    # Find Max Pain from Option Payoffs
    
    minimum_options_payoff = math.inf
    minimum_options_payoff_strike = 0
    
    # loop over dictionary of payoffs for each potential close price
    for (close_price, options_payoff) in total_options_payoff_by_close.items():
        if (options_payoff < minimum_options_payoff):

            # max pain calcs will  filter out pain strikes farther than whis percentage from current price
            # percentage_window = 0.05
            # TODO - must be within 20% of current price
            current_price = stock_info.get_live_price(ticker)
            # if ((abs(current_price) - close_price) < current_price * percentage_window):
            minimum_options_payoff = options_payoff
            minimum_options_payoff_strike = close_price
    
    max_pain = minimum_options_payoff_strike
    # print("max pain is: " + max_pain)

    return max_pain


# ===================================================================

# Loop across all tickers in list, calculating max pain and gap $ / %

tickers = ['NVDA','CMG']

pain_list = []

for ticker in tickers:

    current_price = stock_info.get_live_price(ticker)

    option_chain = op.get_options_chain(ticker)
    calls = option_chain['calls']
    calls = option_chain['puts']
    
    max_pain = get_max_pain(ticker)
    
    gap_dollars = max_pain - current_price
    gap_percentage = gap_dollars / current_price
    
    pain_ranks_dict = {}

    pain_ranks_dict.update({'Ticker': ticker})
    pain_ranks_dict.update({'Price': int(current_price)})
    pain_ranks_dict.update({'Max Pain': max_pain})
    pain_ranks_dict.update({'Gap $': gap_dollars})
    pain_ranks_dict.update({'Gap %': gap_percentage})

    pain_list.append(pain_ranks_dict)      


df = pd.DataFrame(pain_list)
df_sorted = df.sort_values('Gap %')
print(df_sorted )

/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/usr/local/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)
/usr/local/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)
/usr/local/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarnin



Call data for  NVDA
ex:  April 5, 2024
     Strike     Ask  Volume  Open Interest
0     260.0  644.30       6            139
1     300.0  604.35      86             78
2     310.0  594.35       1              2
3     320.0  584.40       2              1
4     330.0  574.40      21             10
5     350.0  554.40      55              1
6     370.0  534.45       1              5
7     390.0  514.45       1              1
8     400.0  504.50       6             24
9     410.0  494.50       6              7
10    420.0  484.50      10             17
11    430.0  474.55       1              2
12    450.0  454.55       6              8
13    460.0  444.55       1             32
14    470.0  434.60       1              4
15    480.0  424.60       3              5
16    490.0  414.60       1              3
17    500.0  404.65      13             19
18    510.0  394.65       8              2
19    520.0  384.65       1             16
20    530.0  374.65       4             10
21    540.0  

/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/usr/local/lib/python3.11/site-



Call data for  CMG
ex:  April 5, 2024
     Strike     Ask Volume Open Interest
0    2020.0  896.00      1             1
1    2040.0    0.00      -             -
2    2060.0  856.00      1             1
3    2080.0  836.00      1             1
4    2150.0  764.40      1             0
5    2200.0  716.00     13             1
6    2250.0  666.00      -             3
7    2260.0  656.00     13             1
8    2280.0  636.00      1             1
9    2290.0  626.00     10            10
10   2300.0  616.00      -             1
11   2320.0  596.00      1             1
12   2400.0  516.00      2             2
13   2450.0  466.90     37            40
14   2455.0  486.30      -             -
15   2500.0  416.00      1             2
16   2540.0  377.20      -             1
17   2545.0  354.00      1             0
18   2550.0  368.00      -             1
19   2560.0  358.00      2             2
20   2575.0  342.00      1             1
21   2585.0  332.00     25            40
22   2590.0  328.

/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
/usr/local/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]
